In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
import pandas as pd
from fastai.tabular import *

In [592]:
#jpeg folder, train.csv and test.csv should in the path which will be mentioned below
path = "~/Machine_learning/data/"

In [593]:
train_info = pd.read_csv(path + 'jpeg/train.csv')

In [ ]:
train_info.head()

In [ ]:
train_info.isna().any()

In [ ]:
#drop all rows having NaN value
train_info = train_info.dropna()
train_info.isna().any()

In [ ]:
train_image = ImageList.from_df(train_info, path=path, folder='jpeg/train', suffix='.jpg')

In [ ]:
# used partial data. Should be removed in high performance machine
train_image = train_image.use_partial_data(sample_pct=0.01)

In [ ]:
cols = ['target']

In [ ]:
np.random.seed(42)
src = (train_image
       .split_by_rand_pct(0.2)
       .label_from_df(cols=cols))

In [ ]:
tfms = get_transforms(max_lighting=0.1, max_zoom=1.05, max_warp=0.)

In [ ]:
data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)


In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.recorder.plot_losses()

In [ ]:
lr = 0.01

In [ ]:
learn.fit_one_cycle(2, lr)

In [ ]:
learn.save('stage-1-rn34')

In [ ]:
learn.export()

In [ ]:
learn.load('stage-1-rn34')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 0.02

In [ ]:
learn.fit_one_cycle(1, slice(1e-5, lr/5))

In [ ]:
learn.export()

In [ ]:
learn.show_results()

In [ ]:
test_info = pd.read_csv(path + "jpeg/test.csv")

In [ ]:
test_data = ImageList.from_df(test_info, path=path, folder='jpeg/test', suffix='.jpg')

In [ ]:
# used partial data. Should be removed in high performance machine
test_data= test_data.use_partial_data(sample_pct=0.001)
len(test_data)

In [ ]:
learn = load_learner(path, test=test_data)

In [ ]:
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
acc = accuracy(preds, y)

In [ ]:
acc

In [ ]:
#thresh = 0.2
#labelled_preds = []
#for pred in preds:
#    for i,p in enumerate(pred):
#        if p > thresh:
#            labelled_preds.append(learn.data.classes[i])
#labelled_preds

In [ ]:
preds

In [ ]:
thresh = 0.5
labelled_preds = []
for pred in preds:
    for i,p in enumerate(pred):
        if p > thresh:
            labelled_preds.append(learn.data.classes[i])

In [ ]:
labelled_preds

In [ ]:
#interp = ClassificationInterpretation.from_learner(learn, ds_type=DatasetType.Train)

In [ ]:
#interp.plot_confusion_matrix()

In [ ]:
fnames = [f[:-4] for f in learn.data.test_ds.items]

In [ ]:
df = pd.DataFrame({'image_name':fnames, 'tags':labelled_preds}, columns=['image_name', 'tags'])


In [ ]:
df.to_csv(path + 'submission.csv', index=False)
